In [2]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - LangChain 구성요소(예: ChatOpenAI, Runnable 계열)를 초기화/사용합니다.
#  - .env에서 OPENAI_API_KEY 등 환경 변수를 로드/사용합니다.
#  - System/Human/AI/Tool 메시지 타입으로 대화를 구성합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: 'from langchain_openai import ChatOpenAI'
# ========================= 코드 해설 주석 끝 ===========================
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


#openai_api_key가져오기
from openai import OpenAI  # 주석처리
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기
#openai_api_key가져오기

llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈어요! 당신은 어떻게 지내세요? 도움이 필요하시면 언제든지 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx6gCS674apVTdSCzDawPrrDJMsRe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f07f238e-204a-4b03-b1dd-84559cde1f66-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - LangChain 구성요소(예: ChatOpenAI, Runnable 계열)를 초기화/사용합니다.
#  - 툴(@tool) 혹은 도구 바인딩/툴콜 처리 로직이 포함되어 있습니다.
#  - 시간/타임존 관련 유틸리티(datetime/pytz)를 사용합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: 'from langchain_core.tools import tool'
# ========================= 코드 해설 주석 끝 ===========================
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [4]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - 툴(@tool) 혹은 도구 바인딩/툴콜 처리 로직이 포함되어 있습니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: '# 도구를 tools 리스트에 추가하고, tool_dict에도 추가'
# ========================= 코드 해설 주석 끝 ===========================
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [5]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - LangChain 구성요소(예: ChatOpenAI, Runnable 계열)를 초기화/사용합니다.
#  - System/Human/AI/Tool 메시지 타입으로 대화를 구성합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: 'from langchain_core.messages import SystemMessage'
# ========================= 코드 해설 주석 끝 ===========================
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dVjyEwPG0etGN7kC85HH11QC', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx6gJJVzs52gEMyh5xDvKoElFOpdI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--47fd8346-25aa-4625-b2fc-780f1299cea8-0', tool_ca

In [6]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - 툴(@tool) 혹은 도구 바인딩/툴콜 처리 로직이 포함되어 있습니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: 'for tool_call in response.tool_calls:'
# ========================= 코드 해설 주석 끝 ===========================
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2025-07-25 15:59:03 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dVjyEwPG0etGN7kC85HH11QC', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx6gJJVzs52gEMyh5xDvKoElFOpdI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--47fd8346-25aa-4625-b2fc-780f1299cea8-0', tool_

In [7]:
# ========================= 코드 해설 주석 시작 =========================
# 이 셀의 역할(요약):
#  - 이 셀은 유틸리티/설정/임포트/헬퍼 함수 등 보조 로직을 수행합니다.
# 주의: 아래 주석은 원본 코드의 동작을 변경하지 않으며, 가독성을 위해 상단에만 추가되었습니다.
# 첫 줄 미리보기: 'llm_with_tools.invoke(messages)'
# ========================= 코드 해설 주석 끝 ===========================
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 2025년 7월 25일 15시 59분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 192, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx6gO0SwhEiQ7C7TvCPx2tQAW6PDT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9571032a-28df-4199-a2f3-7b3166d4f855-0', usage_metadata={'input_tokens': 192, 'output_tokens': 23, 'total_tokens': 215, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})